In [1]:
from IPython import display
%matplotlib inline
import cv2
import gym
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys, shutil
import torch
import omegaconf
import time
import torch

import mbrl.env.cartpole_continuous as cartpole_env
import mbrl.env.reward_fns as reward_fns
import mbrl.env.termination_fns as termination_fns
import mbrl.models as models
import mbrl.planning as planning
import mbrl.util.common as common_util
import mbrl.util as util
from mbrl.util.math import euler_to_quaternion, quaternion_rotation_matrix

import tactile_gym.rl_envs
from tactile_gym.sb3_helpers.params import import_parameters

%load_ext autoreload
%autoreload 2

mpl.rcParams.update({"font.size": 16})

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
# produce a display to render image
from pyvirtualdisplay import Display
_display = Display(visible=False, size=(1400, 900))
_ = _display.start()

# Setup Training Environment and Parameters

In [3]:
# Make the pushing environment
algo_name = 'ppo'
env_name = 'object_push-v0'
rl_params, algo_params, augmentations = import_parameters(env_name, algo_name)
rl_params["env_modes"][ 'observation_mode'] = 'tactile_pose_goal_excluded_data'
rl_params["env_modes"][ 'control_mode'] = 'TCP_position_control'
# rl_params["env_modes"]['movement_mode'] = 'TxTyRz'
rl_params["max_ep_len"] = 2000
rl_params["env_modes"][ 'terminate_early']  = True
rl_params["env_modes"][ 'use_contact'] = True
rl_params["env_modes"][ 'traj_type'] = 'point'
rl_params["env_modes"][ 'task'] = "goal_pos"
rl_params["env_modes"]['additional_reward_settings'] = 'john_guide_off'
rl_params["env_modes"]['mpc_goal_orn_update'] = False
rl_params["env_modes"]['planar_states'] = True
rl_params["env_modes"]['importance_obj_goal_pos'] = 1.0
rl_params["env_modes"]['importance_obj_goal_orn'] = 1.5
rl_params["env_modes"]['importance_tip_obj_orn'] = 1.0

# set limits and goals
TCP_lims = np.zeros(shape=(6, 2))
TCP_lims[0, 0], TCP_lims[0, 1] = -0.1, 0.3  # x lims
TCP_lims[1, 0], TCP_lims[1, 1] = -0.3, 0.3  # y lims
TCP_lims[2, 0], TCP_lims[2, 1] = -0.0, 0.0  # z lims
TCP_lims[3, 0], TCP_lims[3, 1] = -0.0, 0.0  # roll lims
TCP_lims[4, 0], TCP_lims[4, 1] = -0.0, 0.0  # pitch lims
TCP_lims[5, 0], TCP_lims[5, 1] = -180 * np.pi / 180, 180 * np.pi / 180  # yaw lims

# goal parameter
# goal_edges = [(0, -1), (0, 1), (1, 0)] # Top bottom and stright
goal_edges = [(1, 0)]
goal_x_max = np.float64(TCP_lims[0, 1] * 0.9).item()
goal_x_min = 0.0 # np.float64(TCP_lims[0, 0] * 0.6).item()
goal_y_max = np.float64(TCP_lims[1, 1] * 0.6).item()
goal_y_min = np.float64(TCP_lims[1, 0] * 0.6).item()
goal_ranges = [goal_x_min, goal_x_max, goal_y_min, goal_y_max]

rl_params["env_modes"]['tcp_lims'] = TCP_lims.tolist()
rl_params["env_modes"]['goal_edges'] = goal_edges
rl_params["env_modes"]['goal_ranges'] = goal_ranges

print(rl_params)
print(algo_params)

env_kwargs={
    'show_gui':False,
    'show_tactile':False,
    'max_steps':rl_params["max_ep_len"],
    'image_size':rl_params["image_size"],
    'env_modes':rl_params["env_modes"],
}
env = gym.make(env_name, **env_kwargs)

seed = 0
env.seed(seed)
rng = np.random.default_rng(seed=0)
generator = torch.Generator(device=device)
generator.manual_seed(seed)
obs_shape = env.observation_space.shape
act_shape = env.action_space.shape

# This functions allows the model to evaluate the true rewards given an observation 
reward_fn = reward_fns.cartpole
# This function allows the model to know if an observation should make the episode end
term_fn = termination_fns.cartpole

# Define model working directorys
work_dir = os.path.join(os.getcwd(), 'saved_model')
# work_dir = os.path.join(os.getcwd(), 'training_cfg')
if not os.path.exists(work_dir):
    os.mkdir(work_dir)
else:
    for filename in os.listdir(work_dir):
        file_path = os.path.join(work_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

{'algo_name': 'ppo', 'env_name': 'object_push-v0', 'max_ep_len': 2000, 'image_size': [128, 128], 'env_modes': {'movement_mode': 'TyRz', 'control_mode': 'TCP_position_control', 'rand_init_orn': False, 'rand_obj_mass': False, 'traj_type': 'point', 'observation_mode': 'tactile_pose_goal_excluded_data', 'reward_mode': 'dense', 'terminate_early': True, 'use_contact': True, 'task': 'goal_pos', 'additional_reward_settings': 'john_guide_off', 'mpc_goal_orn_update': False, 'planar_states': True, 'importance_obj_goal_pos': 1.0, 'importance_obj_goal_orn': 1.5, 'importance_tip_obj_orn': 1.0, 'tcp_lims': [[-0.1, 0.3], [-0.3, 0.3], [-0.0, 0.0], [-0.0, 0.0], [-0.0, 0.0], [-3.141592653589793, 3.141592653589793]], 'goal_edges': [(1, 0)], 'goal_ranges': [0.0, 0.27, -0.18, 0.18]}, 'policy': 'MultiInputPolicy', 'seed': 1, 'n_stack': 1, 'total_timesteps': 1000000, 'n_eval_episodes': 10, 'n_envs': 10, 'eval_freq': 2000.0}
{'policy_kwargs': {'features_extractor_class': <class 'tactile_gym.sb3_helpers.custom.

pybullet build time: Mar  8 2021 17:26:24


ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [4]:
print(env.reset())
print(env.get_observation())
print(type(env.get_observation()))
# print(env.get_tactile_pose_obs_goal_exluded())

[0.000937 0.000079 0.000006 1.000000 -0.001000 0.000000 0.000000 1.000000]
[0.000937 0.000079 0.000006 1.000000 -0.001000 0.000000 0.000000 1.000000]
<class 'numpy.ndarray'>


In [5]:
print(env.observation_space)
print(env.action_space)
print(obs_shape)
print(act_shape)

Box(8,)
Box(2,)
(8,)
(2,)


In [6]:
trial_length = env._max_steps
num_trials = 1
ensemble_size = 5
initial_buffer_size = 2000
buffer_size = num_trials * trial_length
target_normalised = True

# Everything with "???" indicates an option with a missing value.
# Our utility functions will fill in these details using the 
# environment information
cfg_dict = {
    # dynamics model configuration
    "dynamics_model": {
        "_target_": "mbrl.models.GaussianMLP",
        "device": device,
        "num_layers": 3,
        "ensemble_size": ensemble_size,
        "hid_size": 200,
        "in_size": "???",
        "out_size": "???",
        "deterministic": False,
        "propagation_method": "fixed_model",
        # can also configure activation function for GaussianMLP
        "activation_fn_cfg": {
            "_target_": "torch.nn.LeakyReLU",
            "negative_slope": 0.01
        }
    },
    # options for training the dynamics model
    "algorithm": {
        "learned_rewards": False,
        "target_is_delta": True,
        "normalize": True,
        "target_normalize": target_normalised,
        "dataset_size": buffer_size
    },
    # these are experiment specific options
    "overrides": {
        "trial_length": trial_length,
        "num_steps": num_trials * trial_length,
        "model_batch_size": 32,
        "validation_ratio": 0.05
    }
}
cfg = omegaconf.OmegaConf.create(cfg_dict)

In [7]:
# Create a 1-D dynamics model for this environment
dynamics_model = common_util.create_one_dim_tr_model(cfg, obs_shape, act_shape)

# Create a gym-like environment to encapsulate the model
model_env = models.ModelEnvPushing(env, dynamics_model, termination_fn=None, reward_fn=None, generator=generator)

print(dynamics_model)

/home/qt21590/anaconda3/envs/tactile_gym_mbrl/lib/python3.9/site-packages/hydra/utils.py:32: UserWarning: `OmegaConf.is_none()` is deprecated, see https://github.com/omry/omegaconf/issues/547
  if OmegaConf.is_none(config):


OneDTransitionRewardModel(
  (model): GaussianMLP(
    (hidden_layers): Sequential(
      (0): Sequential(
        (0): EnsembleLinearLayer(num_members=5, in_size=10, out_size=200, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
      )
      (1): Sequential(
        (0): EnsembleLinearLayer(num_members=5, in_size=200, out_size=200, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
      )
      (2): Sequential(
        (0): EnsembleLinearLayer(num_members=5, in_size=200, out_size=200, bias=True)
        (1): LeakyReLU(negative_slope=0.01)
      )
    )
    (mean_and_logvar): EnsembleLinearLayer(num_members=5, in_size=200, out_size=16, bias=True)
  )
)


In [8]:
optimizer_type = "cem"

if optimizer_type == "cem":
    optimizer_cfg = {
            "_target_": "mbrl.planning.CEMOptimizer",
            "num_iterations": 5,
            "elite_ratio": 0.1,
            "population_size": 500,
            "alpha": 0.1,
            "device": device,
            "lower_bound": "???",
            "upper_bound": "???",
            "return_mean_elites": True,
            "clipped_normal": False
        }
elif optimizer_type == "mppi":
    optimizer_cfg = {
            "_target_": "mbrl.planning.MPPIOptimizer",
            "num_iterations": 5,
            "gamma": 1.0,
            "population_size": 500,
            "sigma": 0.95,
            "beta": 0.7,
            "lower_bound": "???",
            "upper_bound": "???",
            "device": device,
        }

elif optimizer_type == "icem":
    optimizer_cfg = {
            "_target_": "mbrl.planning.ICEMOptimizer",
            "num_iterations": 5,
            "elite_ratio": 0.1,
            "population_size": 500,
            "population_decay_factor": 1.25,
            "colored_noise_exponent": 2.0,
            "keep_elite_frac": 0.1,
            "alpha": 0.1,
            "lower_bound": "???",
            "upper_bound": "???",
            "device": device,
        }
else:
    raise ValueError


agent_cfg = omegaconf.OmegaConf.create({
    # this class evaluates many trajectories and picks the best one
    "_target_": "mbrl.planning.TrajectoryOptimizerAgent",
    "planning_horizon": 40,
    "replan_freq": 1,
    "verbose": False,
    "action_lb": "???",
    "action_ub": "???",
    # this is the optimizer to generate and choose a trajectory
    "optimizer_cfg": optimizer_cfg
})

agent = planning.create_trajectory_optim_agent_for_model(
    model_env,
    agent_cfg,
    num_particles=20
)

In [9]:
# Saving config files
config_filename = 'cfg_dict'
config_dir = os.path.join(work_dir, config_filename)
omegaconf.OmegaConf.save(config=cfg, f=config_dir) 
loaded = omegaconf.OmegaConf.load(config_dir)
assert cfg == loaded

agent_config_filename = 'agent_cfg'
agent_config_dir = os.path.join(work_dir, agent_config_filename)
omegaconf.OmegaConf.save(config=agent_cfg, f=agent_config_dir) 
loaded = omegaconf.OmegaConf.load(agent_config_dir)
assert agent_cfg == loaded

env_kwargs_filename = 'env_kwargs'
env_kwargs_dir = os.path.join(work_dir, env_kwargs_filename)
omegaconf.OmegaConf.save(config=env_kwargs, f=env_kwargs_dir) 
loaded = omegaconf.OmegaConf.load(env_kwargs_dir)
assert env_kwargs == loaded

# Environment Tests

In [10]:
env.reset()
print("Current goal index", env.targ_traj_list_id)
print("Position goal trajectory", env.traj_pos_workframe)
# print("Orientation goal trajectory", env.traj_rpy_workframe)
# print("Orientation goal trajectory", env.traj_orn_workframe)

# All of this can be accessed through model_env.env
print(model_env.env.targ_traj_list_id)
print(model_env.env.traj_pos_workframe == env.traj_pos_workframe)
print(model_env.env.goal_pos_workframe)

Current goal index 0
Position goal trajectory [[0.270000 0.149340 0.000000]]
0
[[ True  True  True]]
[0.270000 0.149340 0.000000]


In [11]:
# ----------------- Test termination function in model env -------------------------
early_termination = env.terminate_early
def termination(act: torch.Tensor, next_obs: torch.Tensor, rewards:  torch.Tensor) -> torch.Tensor:
    '''
    Criteria for terminating an episode. Should return a vector of dones of size 
    population_size x batch_size
    '''

    if 'reduced' in env.observation_mode: 
        tcp_pos_to_goal_workframe = next_obs[:, 0:3]
        # tcp_orn_to_goal_workframe = next_obs[:, 3:7]
        # tcp_lin_vel_workframe = next_obs[:, 7:10]
        # tcp_ang_vel_workframe = next_obs[:, 10:13]
        cur_obj_pos_to_goal_workframe = next_obs[:, 13:16]
        # cur_obj_orn_to_goal_workframe = next_obs[:, 16:20]
        # cur_obj_lin_vel_workframe = next_obs[:, 20:23]
        # cur_obj_ang_vel_workframe = next_obs[:, 23:26]

        tcp_pos_workframe = tcp_pos_to_goal_workframe + goal_pos_workframe_batch
        cur_obj_pos_workframe = cur_obj_pos_to_goal_workframe + goal_pos_workframe_batch

        # Calculate distance between goal and current positon
        obj_goal_pos_dist = torch.linalg.norm(cur_obj_pos_to_goal_workframe, axis=1)
    
    elif env.observation_mode == 'tactile_pose_data': 

        if env.planar_states == True: 
            # tcp_pos_to_goal_workframe = torch.zeros((len(next_obs), 3), dtype=torch.float32).to(device)
            tcp_orn_to_goal_workframe = torch.zeros((len(next_obs), 4), dtype=torch.float32).to(device)
            cur_obj_pos_to_goal_workframe = torch.zeros((len(next_obs), 3), dtype=torch.float32).to(device)
            cur_obj_orn_to_goal_workframe = torch.zeros((len(next_obs), 4), dtype=torch.float32).to(device)

            # tcp_pos_to_goal_workframe[:, 0:2] = next_obs[:, 0:2]
            tcp_orn_to_goal_workframe[:, 2:4] = next_obs[:, 0:2]
            cur_obj_pos_to_goal_workframe[:, 0:2]= next_obs[:, 2:4]
            cur_obj_orn_to_goal_workframe[:, 2:4] = next_obs[:, 4:6]
        else:
            # tcp_pos_to_goal_workframe = next_obs[:, 0:3]
            tcp_orn_to_goal_workframe = next_obs[:, 0:4]
            cur_obj_pos_to_goal_workframe = next_obs[:, 4:7]
            cur_obj_orn_to_goal_workframe = next_obs[:, 7:11]

        # Only take in the x and y coordinates
        # tcp_pos_workframe = tcp_pos_to_goal_workframe[:, 0:2] + goal_pos_workframe_batch[:, 0:2]
        cur_obj_pos_workframe = cur_obj_pos_to_goal_workframe[:, 0:2] + goal_pos_workframe_batch[:, 0:2]

        # Calculate distance between goal and current positon
        obj_goal_pos_dist = torch.linalg.norm(cur_obj_pos_to_goal_workframe, axis=1)
            
    elif env.observation_mode == 'tactile_pose_goal_excluded_data': 

        if env.planar_states == True:
            tcp_pos_workframe = torch.zeros((len(next_obs), 3), dtype=torch.float64).to(device)
            tcp_orn_workframe = torch.zeros((len(next_obs), 4), dtype=torch.float64).to(device)
            cur_obj_pos_workframe = torch.zeros((len(next_obs), 3), dtype=torch.float64).to(device)
            cur_obj_orn_workframe = torch.zeros((len(next_obs), 4), dtype=torch.float64).to(device)

            tcp_pos_workframe[:, 0:2] = next_obs[:, 0:2]
            tcp_orn_workframe[:, 2:4] = next_obs[:, 2:4]
            cur_obj_pos_workframe[:, 0:2]= next_obs[:, 4:6]
            cur_obj_orn_workframe[:, 2:4] = next_obs[:, 6:8]
        else:   

            tcp_pos_workframe = next_obs[:, 0:3]
            tcp_orn_workframe = next_obs[:, 3:7]
            cur_obj_pos_workframe = next_obs[:, 7:10]
            cur_obj_orn_workframe = next_obs[:, 10:14]

        # Calculate distance between goal and current positon
        obj_goal_pos_dist = torch.linalg.norm(cur_obj_pos_workframe - goal_pos_workframe_batch, axis=1)

        # calculate tcp to object orn
        tcp_to_obj_orn = model_env.get_orn_dist(cur_obj_orn_workframe, tcp_orn_workframe)

        tip_to_obj_pos = torch.linalg.norm(tcp_pos_workframe - cur_obj_pos_workframe, axis=1)

    else:
        tcp_pos_workframe = next_obs[:, 0:3]
        # tcp_rpy_workframe = next_obs[:, 3:6]
        # tcp_lin_vel_workframe = next_obs[:, 6:9]
        # tcp_ang_vel_workframe = next_obs[:, 9:12]
        cur_obj_pos_workframe = next_obs[:, 12:15]
        # cur_obj_rpy_workframe = next_obs[:, 15:18]
        # cur_obj_lin_vel_workframe = next_obs[:, 18:21]
        # cur_obj_ang_vel_workframe = next_obs[:, 21:24]
        # pred_goal_pos_workframe = next_obs[:, 24:27]
        # pred_goal_rpy_workframe = next_obs[:, 27:30]

        # Calculate distance between goal and current positon
        obj_goal_pos_dist = torch.linalg.norm(cur_obj_pos_workframe - goal_pos_workframe_batch, axis=1)
    
    # print('Object position, ', cur_obj_pos_workframe)
    # print('TCP position, ', tcp_pos_workframe)
    # print(obj_goal_pos_dist)

    # Set obj to goal to smaller than tolerance for testing
    # obj_goal_pos_dist[1] = 0.001

    # intiailise terminated vector
    terminated = torch.zeros((batch_size, 1), dtype=bool).to(device)

    # print("goal position batch before update \n", goal_pos_workframe_batch)
    # print("goal index batch before update", targ_traj_list_id_batch)

    # Early termination if outside of the tcp limits
    if early_termination:
        outside_tcp_lims_idx = outside_tcp_lims(cur_obj_pos_workframe, tcp_to_obj_orn, tip_to_obj_pos)
        terminated[outside_tcp_lims_idx] = True
        rewards[outside_tcp_lims_idx] += env.terminated_early_penalty
        # print("Outside TPC_lims, ", outside_tcp_lims(tcp_pos_workframe, cur_obj_pos_workframe, tip_to_obj_pos))

    # update goals index if subgoal reached
    targ_traj_list_id_batch[obj_goal_pos_dist < model_env.termination_pos_dist] += 1

    # Terminated is true if last subgoal is reached
    terminated[targ_traj_list_id_batch >= model_env.traj_n_points] = True
    rewards[targ_traj_list_id_batch >= model_env.traj_n_points] += model_env.reached_goal_reward

    # Update goal position batch for none terminated samples
    goal_pos_workframe_batch[~terminated[:,0]] = traj_pos_workframe[targ_traj_list_id_batch[~terminated[:,0]]]

    # print("obj to goal distance", obj_goal_pos_dist)
    # print("goal index batch", targ_traj_list_id_batch)
    # print("terminated batch", terminated)
    # print("goal position index not terminated", targ_traj_list_id_batch[~terminated[:,0]])
    # print("The none terminated goals to be updated", traj_pos_workframe[targ_traj_list_id_batch[~terminated[:,0]]])
    # print("The updated goals \n", goal_pos_workframe_batch)
    return terminated

def outside_tcp_lims(cur_obj_pos_workframe, tcp_to_obj_orn, tip_to_obj_pos):
    # xyz_tcp_dist_to_obj = torch.linalg.norm(tcp_pos_workframe - cur_obj_pos_workframe)
    return ((cur_obj_pos_workframe[:, 0] < env.robot.arm.TCP_lims[0,0]) | 
        (cur_obj_pos_workframe[:, 0] > env.robot.arm.TCP_lims[0,1]) | 
        (cur_obj_pos_workframe[:, 1] < env.robot.arm.TCP_lims[1,0]) | 
        (cur_obj_pos_workframe[:, 1] > env.robot.arm.TCP_lims[1,1]) | 
        (tcp_to_obj_orn > model_env.max_tcp_to_obj_orn) | 
        (tip_to_obj_pos > model_env.max_tip_to_obj_pos))
        # (xyz_tcp_dist_to_obj > env.obj_width / 2))                # TODO: exiting episode when roughly lose contact


# Reset environment
batch_size = 3
env.reset()
model_env.reset_batch_goals(batch_size)

# Create goal batches (access through model_env)
traj_pos_workframe = model_env.traj_pos_workframe.clone()
goal_pos_workframe_batch = model_env.goal_pos_workframe_batch.clone()
targ_traj_list_id_batch = model_env.targ_traj_list_id_batch
# targ_traj_list_id_batch = torch.from_numpy(targ_traj_list_id_batch).long()
# targ_traj_list_id_batch[0] = 11

tcp_pos = torch.tensor([[0.1, 0.01, 0], [0.1, 0.01, 0], [0.1, 0.01, 0]]).to(device)
obj_pos = tcp_pos + torch.tensor([[0.0, 0.1, 0.0], [0.0, 0, 0], [0, 0, 0]]).to(device)
tcp_orn = euler_to_quaternion(torch.tensor([[0, 0, 0], [0, 0, 0], [0, 0, 0]])).to(device)
obj_orn = euler_to_quaternion(torch.tensor([[0, 0, 10/180*np.pi], [0, 0, 10/180*np.pi], [0, 0, 10/180*np.pi]])).to(device)
tip_to_obj_orn = model_env.get_orn_dist(tcp_orn, obj_orn)

obs = torch.randn(batch_size, obs_shape[0]).to(device)
if "reduced" in env.observation_mode:
    obs[:, :3] =  torch.tensor([[0.2, 0.05, 0.1], [0.2, 0.05, 0.1], [0.2, 0.05, 0.1]]) 
    obs[:, 13:16] = torch.tensor([[0.2, 0.05, 0.1], [0.2, 0.1, 0.1], [0.2, 0.05, 0.1]])
elif env.observation_mode =='tactile_pose_data':
    if env.planar_states == True:
        # obs[:, 0:2] =  torch.tensor([[0.20, 0.01], [0.3, 0.01], [0.2, 0.01]]).to(device) - goal_pos_workframe_batch[:, 0:2]
        obs[:, 2:4] = torch.tensor([[0.4, 0.01], [0.35, 0.1], [0.2, 0.01]]).to(device) - goal_pos_workframe_batch[:, 0:2]
    else:
        # obs[:, 0:2] =  torch.tensor([[0.20, 0.01], [0.3, 0.01], [0.2, 0.01]]).to(device) - goal_pos_workframe_batch[:, 0:2]
        obs[:, 4:6] = torch.tensor([[0.30, 0.01], [0.20, 0.09], [0.2, 0.01]]).to(device) - goal_pos_workframe_batch[:, 0:2]
elif env.observation_mode == 'tactile_pose_goal_excluded_data': 
    if env.planar_states == True:
        obs[:, 0:2] = tcp_pos[:, 0:2]
        obs[:, 4:6] = obj_pos[:, 0:2]
        obs[:, 2:4] = tcp_orn[:, 2:4]
        obs[:, 6:8] = obj_orn[:, 2:4]
    else:
        # obs[:, 0:2] =  torch.tensor([[0.2, 0.01], [0.31, 0.01], [0.2, 0.01]]) 
        obs[:, 4:6] = torch.tensor([[0.31, 0.1], [0.20, 0.01], [0.2, 0.01]])

else:
    obs[:, :3] =  torch.tensor([[0.3, 0.1, 0.1], [0.3, 0.1, 0.1], [0.3, 0.1, 0.1]]) 
    obs[:, 12:15] = torch.tensor([[0.3, 0.1, 0.1], [0.3, 0.1, 0.1], [0.4, 0.1, 0.1]]) 

act = torch.randn(batch_size, 1).to(device)
rewards_test = torch.zeros(batch_size, 1).to(device)
rewards_push = torch.zeros(batch_size, 1).to(device)
print(termination(act, obs, rewards_test))
print(model_env.termination(act, obs, rewards_push))
print(rewards_test)
print(rewards_push)

tensor([[ True],
        [False],
        [False]], device='cuda:0')
tensor([[ True],
        [False],
        [False]], device='cuda:0')
tensor([[-100.],
        [   0.],
        [   0.]], device='cuda:0')
tensor([[-100.],
        [   0.],
        [   0.]], device='cuda:0')


In [12]:
# ---------------- Test early termination ------------------------
early_termination = True

tcp_pos_workframe = torch.tensor([[0.1, 0.01, 0], [0.1, 0.01, 0]]).to(device)
cur_obj_pos_workframe = tcp_pos_workframe + torch.tensor([[0.1, 0.0, 0.0], [0.0, 0, 0]]).to(device) # The first sample for obj_pos is out of y_lims
tcp_orn = euler_to_quaternion(torch.tensor([[0, 0, 10/180*np.pi], [0, 0, 10/180*np.pi]])).to(device)
obj_orn = euler_to_quaternion(torch.tensor([[0, 0, 10/180*np.pi], [0, 0, 30/180*np.pi]])).to(device)
tip_to_obj_orn = model_env.get_orn_dist(tcp_orn, obj_orn)
tip_to_obj_pos = torch.linalg.norm(tcp_pos_workframe - cur_obj_pos_workframe,  axis=1)
print(outside_tcp_lims(cur_obj_pos_workframe, tip_to_obj_orn, tip_to_obj_pos))

batch_size = 2
env.reset()
model_env.reset_batch_goals(batch_size)

# Create goal batches (access through model_env)
traj_pos_workframe = model_env.traj_pos_workframe
goal_pos_workframe_batch = model_env.goal_pos_workframe_batch
targ_traj_list_id_batch = model_env.targ_traj_list_id_batch

obs = torch.randn(batch_size, 30).to(device)
act = torch.randn(batch_size, 1).to(device)
rewards = torch.randn(batch_size).to(device)
if "reduced" in env.observation_mode:
    obs[:, 0:3] = tcp_pos_workframe
    obs[:, 13:16] = cur_obj_pos_workframe
elif  env.observation_mode == 'tactile_pose_data':
    if env.planar_states == True:
        # obs[:, 0:2] = tcp_pos_workframe[:, 0:2] - goal_pos_workframe_batch[:, 0:2]
        obs[:, 2:4] = cur_obj_pos_workframe[:, 0:2] - goal_pos_workframe_batch[:, 0:2]
    else:
        # obs[:, 0:3] = tcp_pos_workframe - goal_pos_workframe_batch
        obs[:, 4:7] = cur_obj_pos_workframe- goal_pos_workframe_batch  
elif env.observation_mode == 'tactile_pose_goal_excluded_data':
        if env.planar_states == True:
            obs[:, 0:2] = tcp_pos_workframe[:, 0:2]
            obs[:, 2:4] = tcp_orn[:, 2:4]
            obs[:, 4:6] = cur_obj_pos_workframe[:, 0:2]
            obs[:, 6:8] = obj_orn[:, 2:4]
        else:
            # obs[:, 0:3] = tcp_pos_workframe
            obs[:, 4:7] = cur_obj_pos_workframe
else:
    obs[:, 0:3] = tcp_pos_workframe
    obs[:, 12:15] = cur_obj_pos_workframe
print(termination(_, obs, rewards))

tensor([ True, False], device='cuda:0')
tensor([[ True],
        [False]], device='cuda:0')


In [15]:
# --------------- Test reward function  in model env-------------------------------
def xyz_obj_dist_to_goal(cur_obj_pos_workframe):

    # obj to goal distance
    return torch.linalg.norm(cur_obj_pos_workframe - goal_pos_workframe_batch, axis=1)

def get_pos_dist(pos_dist_vector):

    # obj to goal distance
    return torch.linalg.norm(pos_dist_vector, axis=1)

def orn_obj_dist_to_goal_rpy(cur_obj_rpy_workframe):
    cur_obj_orn_workframe = euler_to_quaternion(cur_obj_rpy_workframe)
    inner_product = torch.sum(goal_orn_workframe_batch*cur_obj_orn_workframe, 1)
    return torch.arccos(torch.clip(2 * (inner_product ** 2) - 1, -1, 1))

def orn_obj_dist_to_goal_orn(cur_obj_orn_workframe):
    inner_product = torch.sum(goal_orn_workframe_batch*cur_obj_orn_workframe, 1)
    return torch.arccos(torch.clip(2 * (inner_product ** 2) - 1, -1, 1))

def get_orn_norm(orn_dist_vector):
    """
    Distance between the current obj orientation and goal orientation.
    """
    dist = torch.arccos(torch.clip(
        (2 * (orn_dist_vector[:, 3]**2)) - 1, -1, 1))
    return dist

def get_orn_dist(orn_dist_vector_1, orn_dist_vector_2):
    inner_product = torch.sum(orn_dist_vector_1*orn_dist_vector_2, 1)
    return torch.arccos(torch.clip(2 * (inner_product ** 2) - 1, -1, 1))

def cos_tcp_dist_to_obj(cur_obj_rpy_workframe, tcp_rpy_workframe):
    """
    Cos distance from current orientation of the TCP to the current
    orientation of the object
    """
    
    batch_size = cur_obj_rpy_workframe.shape[0]

    # tip normal to object normal
    cur_obj_orn_workframe = euler_to_quaternion(cur_obj_rpy_workframe)
    obj_rot_matrix_workframe = quaternion_rotation_matrix(cur_obj_orn_workframe)
    obj_rot_matrix_workframe = torch.reshape(obj_rot_matrix_workframe, (batch_size, 3, 3))
    obj_init_vector_workframe = torch.tensor([1.0, 0.0, 0.0], dtype=torch.float32).to(device)
    obj_vector_workframe = torch.matmul(obj_rot_matrix_workframe, obj_init_vector_workframe)
    # obj_vector_workframe = obj_rot_matrix_workframe[:, :, 0]

    tcp_orn_workframe = euler_to_quaternion(tcp_rpy_workframe)
    tip_rot_matrix_workframe = quaternion_rotation_matrix(tcp_orn_workframe)
    tip_rot_matrix_workframe  = torch.reshape(tip_rot_matrix_workframe, (batch_size, 3, 3))
    tip_init_vector_workframe  = torch.tensor([1.0, 0.0, 0.0], dtype=torch.float32).to(device)
    tip_vector_workframe  = torch.matmul(tip_rot_matrix_workframe, tip_init_vector_workframe)
    # tip_vector_workframe = tip_rot_matrix_workframe[:, :, 0]

    obj_tip_dot_product = torch.sum(obj_vector_workframe*tip_vector_workframe, 1)
    cos_sim_workfrfame = obj_tip_dot_product / (
        torch.linalg.norm(obj_vector_workframe, axis=1) * torch.linalg.norm(tip_vector_workframe, axis=1)
    )
    cos_dist_workframe = 1 - cos_sim_workfrfame

    return cos_dist_workframe

def cos_tcp_dist_to_obj_reduced(cur_obj_orn_to_goal_workframe, tcp_orn_to_goal_workframe):
    """
    Cos distance from current orientation of the TCP to the current
    orientation of the object
    """
    
    batch_size = cur_obj_orn_to_goal_workframe.shape[0]

    # tip normal to object normal
    obj_rot_matrix_workframe = quaternion_rotation_matrix(cur_obj_orn_to_goal_workframe)
    obj_rot_matrix_workframe = torch.reshape(obj_rot_matrix_workframe, (batch_size, 3, 3))
    # obj_init_vector_workframe = torch.tensor([1.0, 0.0, 0.0], dtype=torch.float64)
    # obj_vector_workframe = torch.matmul(obj_rot_matrix_workframe, obj_init_vector_workframe)
    obj_vector_workframe = obj_rot_matrix_workframe[:, :, 0]

    tip_rot_matrix_workframe = quaternion_rotation_matrix(tcp_orn_to_goal_workframe)
    tip_rot_matrix_workframe  = torch.reshape(tip_rot_matrix_workframe, (batch_size, 3, 3))
    # tip_init_vector_workframe  = torch.tensor([1.0, 0.0, 0.0], dtype=torch.float64)
    # tip_vector_workframe  = torch.matmul(tip_rot_matrix_workframe, tip_init_vector_workframe)
    tip_vector_workframe = tip_rot_matrix_workframe[:, :, 0]

    obj_tip_dot_product = torch.sum(obj_vector_workframe*tip_vector_workframe, 1)
    cos_sim_workfrfame = obj_tip_dot_product / (
        torch.linalg.norm(obj_vector_workframe, axis=1) * torch.linalg.norm(tip_vector_workframe, axis=1)
    )
    cos_dist_workframe = 1 - cos_sim_workfrfame 

    return cos_dist_workframe

def cos_tcp_Rz_dist_to_obj(cos_cur_obj_Rz_to_goal_workframe, cos_tcp_Rz_to_goal_workframe):
    cos_sim_workframe = torch.cos(
        torch.arccos(cos_tcp_Rz_to_goal_workframe) - torch.arccos(cos_cur_obj_Rz_to_goal_workframe)
        )
    return 1 - cos_sim_workframe

def reward(act: torch.Tensor, next_obs: torch.Tensor) -> torch.Tensor:
    '''
    Caculate the reward given a batch of observations 
    '''

    batch_size = next_obs.shape[0]

    if 'reduced' in env.observation_mode: 
        # tcp_pos_to_goal_workframe = next_obs[:, 0:3]
        tcp_orn_to_goal_workframe = next_obs[:, 3:7]
        # tcp_lin_vel_workframe = next_obs[:, 7:10]
        # tcp_ang_vel_workframe = next_obs[:, 10:13]
        cur_obj_pos_to_goal_workframe = next_obs[:, 13:16]
        cur_obj_orn_to_goal_workframe = next_obs[:, 16:20]
        # cur_obj_lin_vel_workframe = next_obs[:, 20:23]
        # cur_obj_ang_vel_workframe = next_obs[:, 23:26]

        obj_goal_pos_dist = get_pos_dist(cur_obj_pos_to_goal_workframe)
        obj_goal_orn_dist = get_orn_norm(cur_obj_orn_to_goal_workframe)
        tip_obj_orn_dist = cos_tcp_dist_to_obj_reduced(cur_obj_orn_to_goal_workframe, tcp_orn_to_goal_workframe)

        print(obj_goal_orn_dist)
    elif env.observation_mode == 'tactile_pose_data': 
        if env.planar_states == True: 
            # tcp_pos_to_goal_workframe = torch.zeros((len(next_obs), 3), dtype=torch.float32).to(device)
            tcp_orn_to_goal_workframe = torch.zeros((len(next_obs), 4), dtype=torch.float32).to(device)
            cur_obj_pos_to_goal_workframe = torch.zeros((len(next_obs), 3), dtype=torch.float32).to(device)
            cur_obj_orn_to_goal_workframe = torch.zeros((len(next_obs), 4), dtype=torch.float32).to(device)

            # tcp_pos_to_goal_workframe[:, 0:2] = next_obs[:, 0:2]
            tcp_orn_to_goal_workframe[:, 2:4] = next_obs[:, 0:2]
            cur_obj_pos_to_goal_workframe[:, 0:2]= next_obs[:, 2:4]
            cur_obj_orn_to_goal_workframe[:, 2:4] = next_obs[:, 4:6]
        else:
            # tcp_pos_to_goal_workframe = next_obs[:, 0:3]
            tcp_orn_to_goal_workframe = next_obs[:, 0:4]
            cur_obj_pos_to_goal_workframe = next_obs[:, 4:7]
            cur_obj_orn_to_goal_workframe = next_obs[:, 7:11]
            
        obj_goal_pos_dist = get_pos_dist(cur_obj_pos_to_goal_workframe)
        obj_goal_orn_dist = get_orn_norm(cur_obj_orn_to_goal_workframe)
        tip_obj_orn_dist = get_orn_dist(cur_obj_orn_to_goal_workframe, tcp_orn_to_goal_workframe)

    elif env.observation_mode == 'tactile_pose_goal_excluded_data':
        if env.planar_states == True:
            # tcp_pos_workframe = torch.zeros((len(next_obs), 3), dtype=torch.float32).to(device)
            tcp_orn_workframe = torch.zeros((len(next_obs), 4), dtype=torch.float32).to(device)
            cur_obj_pos_workframe = torch.zeros((len(next_obs), 3), dtype=torch.float32).to(device)
            cur_obj_orn_workframe = torch.zeros((len(next_obs), 4), dtype=torch.float32).to(device)

            # tcp_pos_workframe[:, 0:2] = next_obs[:, 0:2]
            tcp_orn_workframe[:, 2:4] = next_obs[:, 2:4]
            cur_obj_pos_workframe[:, 0:2]= next_obs[:, 4:6]
            cur_obj_orn_workframe[:, 2:4] = next_obs[:, 6:8]
        else:   
            # tcp_pos_workframe = next_obs[:, 0:3]
            tcp_orn_workframe = next_obs[:, 0:4]
            cur_obj_pos_workframe = next_obs[:, 4:7]
            cur_obj_orn_workframe = next_obs[:, 7:11]

        obj_goal_pos_dist = xyz_obj_dist_to_goal(cur_obj_pos_workframe)
        obj_goal_orn_dist = orn_obj_dist_to_goal_orn(cur_obj_orn_workframe)
        tip_obj_orn_dist = get_orn_dist(cur_obj_orn_workframe, tcp_orn_workframe)
        
    else:
        # tcp_pos_workframe = next_obs[:, 0:3]
        tcp_rpy_workframe = next_obs[:, 3:6]
        # tcp_lin_vel_workframe = next_obs[:, 6:9]
        # tcp_ang_vel_workframe = next_obs[:, 9:12]
        cur_obj_pos_workframe = next_obs[:, 12:15]
        cur_obj_rpy_workframe = next_obs[:, 15:18]
        # cur_obj_lin_vel_workframe = next_obs[:, 18:21]
        # cur_obj_ang_vel_workframe = next_obs[:, 21:24]
        # pred_goal_pos_workframe = next_obs[:, 24:27]
        # pred_goal_rpy_workframe = next_obs[:, 27:30]

        obj_goal_pos_dist = xyz_obj_dist_to_goal(cur_obj_pos_workframe)
        obj_goal_orn_dist = orn_obj_dist_to_goal_rpy(cur_obj_rpy_workframe)
        tip_obj_orn_dist = cos_tcp_dist_to_obj(cur_obj_rpy_workframe, tcp_rpy_workframe)

    reward = -(
        (env.W_obj_goal_pos * obj_goal_pos_dist)
        + (env.W_obj_goal_orn * obj_goal_orn_dist)
        + (env.W_tip_obj_orn * tip_obj_orn_dist)
        )
    reward = reward[:, None]

    return reward

# Create observation and goal batch 
batch_size = 2
obs = env.reset()
for i in range(1):
    obs, _, done, _ = env.step(env.action_space.sample())
obs = torch.tensor(obs).to(torch.float32).to(device)
model_env.reset_batch_goals(batch_size)

obs_batch = torch.tile(obs, (batch_size,) + tuple([1] * obs.ndim))
goal_pos_workframe_batch = model_env.goal_pos_workframe_batch
goal_orn_workframe_batch = model_env.goal_orn_workframe_batch

print(env.dense_reward())
print(reward(_, obs_batch))
print(model_env.reward(_, obs_batch))

-0.0771899710385694
tensor([[-0.0772],
        [-0.0772]], device='cuda:0')
tensor([[-0.0772],
        [-0.0772]], device='cuda:0')


In [17]:
# check update goal function
batch_size = 2
obs = env.reset()
for i in range(100):
    obs, _, done, _ = env.step(env.action_space.sample())
obs = torch.tensor(obs).to(torch.float32).to(device)
model_env.reset_batch_goals(batch_size)
obs_batch = torch.tile(obs, (batch_size,) + tuple([1] * obs.ndim))

print(env.goal_orn_workframe)
model_env.update_goal_orn(obs_batch)
print(model_env.goal_orn_workframe_batch)

[-0.000001 0.000001 0.104460 0.994529]
tensor([[0.0000, 0.0000, 0.1045, 0.9945],
        [0.0000, 0.0000, 0.1045, 0.9945]], device='cuda:0')


In [19]:
# Test reward and terminal function using next_observ from model.sample()
planning_horizon = agent.optimizer.horizon
# initialise action sequence
action_lb = env.action_space.low.tolist()
action_ub = env.action_space.high.tolist()
initial_solution = (((torch.tensor(action_lb) + torch.tensor(action_ub)) / 2)
            .float().to(device)
        )
initial_solution = initial_solution.repeat((planning_horizon, 1))
mu, dispersion = agent.optimizer.optimizer._init_population_params(initial_solution)
action_sequences = torch.zeros((500,) + initial_solution.shape).to(device)
action_sequences = agent.optimizer.optimizer._sample_population(mu, dispersion, action_sequences)

# Intialise state and create model state for model input
env.reset()
for i in range(100):
    obs, _, done, _ = env.step(env.action_space.sample())
initial_state = obs
tiling_shape = (20 * 500,) + tuple(
    [1] * initial_state.ndim
)
initial_obs_batch = np.tile(initial_state, tiling_shape).astype(np.float32)
model_state = model_env.reset(initial_obs_batch, return_as_np=False)

# get action for time step
action_for_step = action_sequences[:, 0, :]
action_batch = torch.repeat_interleave(
                    action_for_step, 20, dim=0
                )

# Get next observation from model
(
    next_observs,
    pred_rewards,
    pred_terminals,
    next_model_state,
) = model_env.dynamics_model.sample(
    action_batch,
    model_state,
    deterministic=False,
    rng=model_env._rng,
)

# Next obervation types
# print(next_observs.type())
# print(next_observs.dtype)
# print(next_observs.shape)

# Create observation and goal batch 
batch_size = next_observs.shape[0]
model_env.reset_batch_goals(batch_size)

# Get global variables needed for reward function
goal_pos_workframe_batch = model_env.goal_pos_workframe_batch
goal_orn_workframe_batch = model_env.goal_orn_workframe_batch

print(reward(_, next_observs))
print(model_env.reward(_, next_observs))
print(any((reward(_, next_observs) == model_env.reward(_, next_observs))))

# Get the global variables needed for termination function
traj_pos_workframe = model_env.traj_pos_workframe
targ_traj_list_id_batch = model_env.targ_traj_list_id_batch
random_reward = reward(_, next_observs)

# print(termination(_, next_observs))
# print(model_env.termination(_, next_observs))
# print(model_env.termination_fn(act, next_observs))
print(all((termination(_, next_observs, random_reward) == model_env.termination(_, next_observs, random_reward))))
# print(random_reward)


tensor([[-6.7497],
        [-1.2274],
        [-1.4438],
        ...,
        [-0.0000],
        [-6.4915],
        [-0.0000]], device='cuda:0', grad_fn=<UnsqueezeBackward0>)
tensor([[-6.7497],
        [-1.2274],
        [-1.4438],
        ...,
        [-0.0000],
        [-6.4915],
        [-0.0000]], device='cuda:0', grad_fn=<UnsqueezeBackward0>)
True
True


In [20]:
# Test evaluation_action_sequences
planning_horizon = agent.optimizer.horizon
# initialise action sequence
# action_lb = env.action_space.low.tolist()
# action_ub = env.action_space.high.tolist()
# initial_solution = (((torch.tensor(action_lb) + torch.tensor(action_ub)) / 2)
#             .float()
#         )
# initial_solution = initial_solution.repeat((15, 1))
# mu, dispersion = agent.optimizer.optimizer._init_population_params(initial_solution)
# action_sequences = torch.zeros((500,) + initial_solution.shape)
# action_sequences = agent.optimizer.optimizer._sample_population(mu, dispersion, action_sequences)
# # print(initial_solution.shape)

# create random action sequences
initial_solution = torch.from_numpy(np.array([env.action_space.sample() for _ in range(planning_horizon)])).float().to(device)
# print(initial_solution.shape)
mu, dispersion = agent.optimizer.optimizer._init_population_params(initial_solution)
action_sequences = torch.zeros((500,) + initial_solution.shape).to(device)
action_sequences = agent.optimizer.optimizer._sample_population(mu, dispersion, action_sequences)

# Initialise environment
initial_state = env.reset()

# evaluate sequences
print(model_env.evaluate_action_sequences(action_sequences, initial_state, 20).shape)
print(any(model_env.targ_traj_list_id_batch!=1))

torch.Size([500])
True


# Training 

In [ ]:
def get_states_from_obs(obs):
    if "goal_excluded" in env.observation_mode:
        if env.planar_states == True:
            tcp_pos_workframe = np.zeros(3)
            tcp_orn_workframe = np.zeros(4)
            cur_obj_pos_workframe = np.zeros(3)
            cur_obj_orn_workframe = np.zeros(4)

            tcp_pos_workframe[0:2] = obs[0:2]
            tcp_orn_workframe[2:4] = obs[2:4]
            cur_obj_pos_workframe[0:2]= obs[4:6]
            cur_obj_orn_workframe[2:4] = obs[6:8]
        else:   
            # tcp_pos_workframe = obs[0:3]
            # tcp_orn_workframe = obs[0:4]
            cur_obj_pos_workframe = obs[4:7]
            # cur_obj_orn_workframe = obs[7:11]

    else:
        if env.planar_states == True: 
            # tcp_pos_to_goal_workframe = np.zeros(3)
            # tcp_orn_to_goal_workframe = np.zeros(4)
            cur_obj_pos_to_goal_workframe = np.zeros(3)
            # cur_obj_orn_to_goal_workframe = np.zeros(4)

            # tcp_pos_to_goal_workframe[0:2] = obs[0:2]
            # tcp_orn_to_goal_workframe[2:4] = obs[0:2]
            cur_obj_pos_to_goal_workframe[0:2]= obs[2:4]
            # cur_obj_orn_to_goal_workframe[2:4] = obs[4:6]
        else:
            # tcp_pos_to_goal_workframe = obs[0:3]
            # tcp_orn_to_goal_workframe = obs[0:4]
            cur_obj_pos_to_goal_workframe = obs[4:7]
            # cur_obj_orn_to_goal_workframe = obs[7:11]

        # tcp_pos_workframe = obs[0:3] + env.goal_pos_workframe
        cur_obj_pos_workframe = cur_obj_pos_to_goal_workframe + env.goal_pos_workframe

    return tcp_pos_workframe, tcp_orn_workframe, cur_obj_pos_workframe, cur_obj_orn_workframe

In [ ]:
# # make directory for saving data
# data_columns = ['trial','trial_steps', 'time_steps', 'tcp_x','tcp_y','tcp_z','contact_x', 'contact_y', 'contact_z', 'tcp_Rz', 'contact_Rz', 'goal_x', 'goal_y', 'goal_z', 'rewards', 'contact', 'dones']
# training_result_directory = os.path.join(work_dir, "training_result")
# os.makedirs(training_result_directory, exist_ok=True)

# def save_data(arg):
#     initial_buffer = []
#     (env, obs, action, next_obs, reward, done, info) = arg
#     (tcp_pos_workframe, 
#     tcp_orn_workframe,
#     cur_obj_pos_workframe, 
#     cur_obj_orn_workframe) = get_states_from_obs(env, obs)
#     cur_obj_rpy_workframe = env._pb.getEulerFromQuaternion(cur_obj_orn_workframe)
#     tcp_rpy_workframe = env._pb.getEulerFromQuaternion(cur_obj_orn_workframe)
#     initial_buffer.append(np.hstack([0, 
#                                     0, 
#                                     0,
#                                     tcp_pos_workframe, 
#                                     cur_obj_pos_workframe,
#                                     tcp_rpy_workframe[2],
#                                     cur_obj_rpy_workframe[2],
#                                     env.goal_pos_workframe, 
#                                     reward, 
#                                     info["tip_in_contact"],
#                                     done]))
#     pd.DataFrame(initial_buffer).to_csv(os.path.join(training_result_directory, "{}_result.csv".format("initial_buffer")), mode='a', header=False)


replay_buffer = common_util.create_replay_buffer(cfg, obs_shape, act_shape, rng=rng)
common_util.rollout_agent_trajectories(
    env,
    initial_buffer_size, # initial exploration steps
    planning.RandomAgent(env),
    {}, # keyword arguments to pass to agent.act()
    replay_buffer=replay_buffer,
    trial_length=trial_length,
    # callback=save_data,
)

print("# samples stored", replay_buffer.num_stored)

# df = pd.read_csv(os.path.join(training_result_directory, 'initial_buffer_result.csv'), names = data_columns)
# loss_contact = False
# print(len(df.query("contact==@loss_contact")))

In [ ]:
train_losses = [0.0]
val_scores = [0.0]

def train_callback(_model, _total_calls, _epoch, tr_loss, val_score, _best_val):
    train_losses.append(tr_loss)
    val_scores.append(val_score.mean().item())   # this returns val score per ensemble model
    

In [ ]:
def update_axes(_axs, _frame, _text, _trial, _steps_trial, _plan_time, _train_time, _all_rewards, _goal_reached, _train_loss, _val_loss, force_update=False):
    if not force_update and (_steps_trial % 10 != 0):
        return
    _axs[0].imshow(_frame)
    _axs[0].set_xticks([])
    _axs[0].set_yticks([])
    _axs[0].tick_params(axis='x', colors='white')
    _axs[0].tick_params(axis='y', colors='white')
    _axs[1].clear()
    _axs[1].set_xlim([0, num_trials + .1])
    _axs[1].set_ylim([min(_all_rewards), 0])
    _axs[1].set_xlabel("Trial")
    _axs[1].set_ylabel("Trial reward")
    _axs[1].xaxis.label.set_color('white')
    _axs[1].yaxis.label.set_color('white')
    _axs[1].tick_params(axis='x', colors='white')
    _axs[1].tick_params(axis='y', colors='white')
    _axs[1].plot(_all_rewards, 'bs-', _goal_reached, 'rs')
    _text.set_text("Trial {}: {} steps\nTrain Loss: {:.2f}\nVal Loss: {:.3g}\nPlan time: {:.2f} s/step\nTrain time: {:.2f} s".format(_trial + 1, _steps_trial, _train_loss, _val_loss, _plan_time, _train_time))
    display.display(plt.gcf())  
    display.clear_output(wait=True)

text_kwargs = dict(ha='center', va='center', fontsize=28, color='C1')
def update_axes_text(_axs, _trial, _steps_trial, _plan_time, _train_time, _all_rewards, _goal_reached, _train_loss, _val_loss, force_update=False):
    if not force_update and (_steps_trial % 10 != 0):
        return
    _axs[0].clear()
    _axs[0].text(0.5, 0.5, 
    "Trial {}: {} steps\nTrain Loss: {:.2f}\nVal Loss: {:.3g}\nPlan time: {:.2f} s/step\nTrain time: {:.2f} s".format(_trial + 1, _steps_trial, _train_loss, _val_loss, _plan_time, _train_time), **text_kwargs)
    _axs[1].clear()
    _axs[1].set_xlim([0, num_trials + .1])
    _axs[1].set_ylim([min(_all_rewards), 0])
    _axs[1].set_xlabel("Trial")
    _axs[1].set_ylabel("Trial reward")
    _axs[1].xaxis.label.set_color('white')
    _axs[1].yaxis.label.set_color('white')
    _axs[1].tick_params(axis='x', colors='white')
    _axs[1].tick_params(axis='y', colors='white')
    _axs[1].plot(_all_rewards, 'bs-', _goal_reached, 'rs')
    display.display(plt.gcf())  
    display.clear_output(wait=True)

# Test plot function
env.reset()
fig, axs = plt.subplots(1, 2, figsize=(14, 3.75), gridspec_kw={"width_ratios": [1, 1]})
ax_text = axs[0].text(130, 80, "")
ax_text.set_color('white')
all_rewards = np.random.randint(-500, 0, num_trials + 1)
goal_reached = np.random.randint(-500, 0, num_trials + 1)
steps = np.sort(np.random.randint(0,10000, len(all_rewards)))
plan_time = np.random.rand()
train_time = np.random.rand()
# update_axes(axs,env.render(mode="rgb_array"),  ax_text, 0, 0, plan_time, train_time, all_rewards, goal_reached, train_losses[-1], val_scores[-1], force_update=True)
update_axes_text(axs, 0, 0, plan_time, train_time, all_rewards, goal_reached, train_losses[-1], val_scores[-1], force_update=True)

In [ ]:
# Create logging folder
print(f"Results will be saved at {work_dir}.")

# Create a trainer for the model
model_trainer = models.ModelTrainer(dynamics_model, optim_lr=1e-3, weight_decay=5e-5)

# Create visualization objects
fig, axs = plt.subplots(1, 2, figsize=(14, 3.75), gridspec_kw={"width_ratios": [1, 1]})
ax_text = axs[0].text(130, 80, "")
ax_text.set_color('white')

# Main PETS loop
all_rewards = [0]
total_steps = [0]
goal_reached = [0]
training_result = []
plan_time = 0.0
train_time = 0.0

record_video = True
record_video_frequency = 5

for trial in range(num_trials):

    # Reset 
    obs = env.reset()    
    agent.reset()
    done = False
    trial_reward = 0.0
    trial_pb_steps = 0.0
    steps_trial = 0

    # Record video
    if record_video and (trial+1) % record_video_frequency == 0:
        record_every_n_frames = 3
        render_img = env.render(mode="rgb_array")
        render_img_size = (render_img.shape[1], render_img.shape[0])
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(
            os.path.join(work_dir, "training_policy_trial_{}.mp4".format((trial+1))),
            fourcc,
            24.0,
            render_img_size,
        )
        
    update_axes_text(axs, trial, steps_trial, plan_time, train_time,
        all_rewards, goal_reached, train_losses[-1], val_scores[-1])
    # update_axes(axs, env.render(mode="rgb_array"), ax_text, trial, steps_trial, plan_time, train_time,
    #     all_rewards, goal_reached, train_losses[-1], val_scores[-1])
    
    tcp_pos_workframe, tcp_orn_workframe, cur_obj_pos_workframe, cur_obj_orn_workframe = get_states_from_obs(obs)
    cur_obj_rpy_workframe = env._pb.getEulerFromQuaternion(cur_obj_orn_workframe)
    tcp_rpy_workframe = env._pb.getEulerFromQuaternion(tcp_orn_workframe)
    training_result.append(np.hstack([trial, 
                                    steps_trial, 
                                    trial_pb_steps,
                                    tcp_pos_workframe, 
                                    cur_obj_pos_workframe,
                                    tcp_rpy_workframe[2],
                                    cur_obj_rpy_workframe[2],
                                    env.goal_pos_workframe, 
                                    trial_reward, 
                                    False,
                                    done]))
    while not done:

        if steps_trial == 0:
            # --------------- Model Training -----------------
            dynamics_model.update_normalizer(replay_buffer.get_all())  # update normalizer stats            
            dataset_train, dataset_val = common_util.get_basic_buffer_iterators(
                replay_buffer,
                batch_size=cfg.overrides.model_batch_size,
                val_ratio=cfg.overrides.validation_ratio,
                ensemble_size=ensemble_size,
                shuffle_each_epoch=True,
                bootstrap_permutes=False,  # build bootstrap dataset using sampling with replacement
            )
            
            start_train_time = time.time()
            model_trainer.train(
                dataset_train, 
                dataset_val=dataset_val, 
                num_epochs=50, 
                patience=50, 
                callback=train_callback,
                silent=True)
            train_time = time.time() - start_train_time

            if work_dir is not None:
                dynamics_model.save(str(work_dir))
                replay_buffer.save(work_dir)

        # --- Doing env step using the agent and adding to model dataset ---
        start_plan_time = time.time()
        next_obs, reward, done, info = common_util.step_env_and_add_to_buffer(
            env, obs, agent, {}, replay_buffer)
        plan_time = time.time() - start_plan_time

        update_axes_text(axs, trial, steps_trial, plan_time, train_time,
            all_rewards, goal_reached, train_losses[-1], val_scores[-1])
        # update_axes(
        #     axs, env.render(mode="rgb_array"), ax_text, trial, steps_trial, plan_time, train_time, 
        #     all_rewards, goal_reached,  train_losses[-1], val_scores[-1])

        obs = next_obs
        trial_reward += reward
        trial_pb_steps += info["num_of_pb_steps"]
        steps_trial += 1

        # Save data for plotting training performance
        tcp_pos_workframe, tcp_orn_workframe, cur_obj_pos_workframe, cur_obj_orn_workframe = get_states_from_obs(obs)
        cur_obj_rpy_workframe = env._pb.getEulerFromQuaternion(cur_obj_orn_workframe)
        tcp_rpy_workframe = env._pb.getEulerFromQuaternion(tcp_orn_workframe)
        training_result.append(np.hstack([trial,
                                        steps_trial,
                                        trial_pb_steps * env._sim_time_step,
                                        tcp_pos_workframe, 
                                        cur_obj_pos_workframe, 
                                        tcp_rpy_workframe[2],
                                        cur_obj_rpy_workframe[2],
                                        env.goal_pos_workframe, 
                                        trial_reward, 
                                        info["tip_in_contact"],
                                        done]))
        
        # Record video at every n trials
        if record_video and (trial+1) % record_video_frequency == 0 and steps_trial % record_every_n_frames == 0:
            render_img = env.render(mode="rgb_array")
            render_img = cv2.cvtColor(render_img, cv2.COLOR_BGR2RGB)
            out.write(render_img)

        if steps_trial == trial_length:
            break

    all_rewards.append(trial_reward)
    total_steps.append(steps_trial + total_steps[-1])

    # save goal reached data during training
    if env.single_goal_reached:
        goal_reached.append(trial_reward)
    else:
        goal_reached.append(0)

    # release video at every n trials
    if record_video and (trial+1) % record_video_frequency == 0:
        out.release()

update_axes_text(axs, trial, steps_trial, plan_time, train_time,
    all_rewards, goal_reached, train_losses[-1], val_scores[-1], force_update=True)
# update_axes(axs, env.render(mode="rgb_array"), ax_text, trial, steps_trial, plan_time, train_time, 
#     all_rewards, goal_reached, train_losses[-1], val_scores[-1], force_update=True)

# Plot results
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(total_steps[1:], all_rewards[1:], 'bs-', total_steps[1:], goal_reached[1:], 'rs')
ax.set_xlabel("Samples")
ax.set_ylabel("Trial reward")
fig.savefig(os.path.join(work_dir, "output.png"))

In [ ]:
def plot_and_save_push_plots(df, trials, directory):
    loss_contact = False
    for trial in range(trials):
        fig_xy, ax = plt.subplots(figsize=(12, 6))
        ax.plot(df.query("trial==@trial")["tcp_x"], df.query("trial==@trial")["tcp_y"], "b-", label='tcp psosition')
        ax.plot(df.query("trial==@trial").query("contact==@loss_contact")["tcp_x"], df.query("trial==@trial").query("contact==@loss_contact")["tcp_y"], "g+", markersize=20)
        ax.plot(df.query("trial==@trial")["contact_x"], df.query("trial==@trial")["contact_y"], "r-", label='contact psosition')
        ax.plot(df.query("trial==@trial").query("contact==@loss_contact")["contact_x"], df.query("trial==@trial").query("contact==@loss_contact")["contact_y"], "gx", markersize=20)
        ax.plot(df.query("trial==@trial")["goal_x"].iloc[0], df.query("trial==@trial")["goal_y"].iloc[0], "x", markersize=20, markeredgecolor="black", label="goal position")
        
        for i, rows in df.query("trial==@trial").iterrows():
            if i % 10 == 0:
                tcp_x, tcp_y, tcp_Rz= rows["tcp_x"], rows["tcp_y"], rows["tcp_Rz"]
                tcp_dx, tcp_dy = 0.05 * np.cos(tcp_Rz), 0.05 * np.sin(tcp_Rz)
                plt.arrow(tcp_x, tcp_y, tcp_dx, tcp_dy, color='b')
                obj_x, obj_y, obj_Rz= rows["contact_x"], rows["contact_y"], rows["contact_Rz"]
                obj_dx, obj_dy = 0.05 * np.cos(obj_Rz), 0.05 * np.sin(obj_Rz)
                plt.arrow(obj_x, obj_y, obj_dx, obj_dy, color='r')
        
        ax.set_xlabel("x workframe")
        ax.set_ylabel("y workframe")
        ax.set_xlim([env.robot.arm.TCP_lims[0, 0], env.robot.arm.TCP_lims[0, 1]])
        ax.set_ylim([env.robot.arm.TCP_lims[1, 0], env.robot.arm.TCP_lims[1, 1]])
        ax.legend()
        fig_xy.savefig(os.path.join(directory, "workframe_plot_trial_{}.png".format(trial)))
        plt.close(fig_xy)

        fig_time_xy, axs = plt.subplots(1, 2, figsize=(14, 3.75), gridspec_kw={"width_ratios": [1, 1]})
        axs[0].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["tcp_x"], "bs", label='tcp ')
        axs[0].plot(df.query("trial==@trial").query("contact==@loss_contact")["time_steps"], df.query("trial==@trial").query("contact==@loss_contact")["tcp_x"], "g+", markersize=20)
        axs[0].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["contact_x"], "rs", label='contact')
        axs[0].plot(df.query("trial==@trial").query("contact==@loss_contact")["time_steps"], df.query("trial==@trial").query("contact==@loss_contact")["contact_x"], "gx", markersize=20)
        axs[0].set_xlabel("Time steps (s)")
        axs[0].set_ylabel("x axis workframe")
        axs[0].set_ylim([env.robot.arm.TCP_lims[0, 0], env.robot.arm.TCP_lims[0, 1]])
        axs[0].legend()
        axs[1].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["tcp_y"], "bs", label='tcp')
        axs[1].plot(df.query("trial==@trial").query("contact==@loss_contact")["time_steps"], df.query("trial==@trial").query("contact==@loss_contact")["tcp_y"], "g+", markersize=20)
        axs[1].plot(df.query("trial==@trial")["time_steps"], df.query("trial==@trial")["contact_y"], "rs", label='contact')
        axs[1].plot(df.query("trial==@trial").query("contact==@loss_contact")["time_steps"], df.query("trial==@trial").query("contact==@loss_contact")["contact_y"], "gx", markersize=20)
        axs[1].set_xlabel("Time steps (s)")
        axs[1].set_ylabel("y axis workframe")
        axs[1].set_ylim([env.robot.arm.TCP_lims[1, 0], env.robot.arm.TCP_lims[1, 1]])
        axs[1].legend()
        fig_time_xy.savefig(os.path.join(directory, "time_plot_trial_{}.png".format(trial)))
        plt.close(fig_time_xy)

# Save data 
training_result = np.array(training_result)
data_columns = ['trial','trial_steps', 'time_steps', 'tcp_x','tcp_y','tcp_z','contact_x', 'contact_y', 'contact_z', 'tcp_Rz', 'contact_Rz', 'goal_x', 'goal_y', 'goal_z', 'rewards', 'contact', 'dones']
df_training = pd.DataFrame(training_result, columns = data_columns)
pd.DataFrame(training_result).to_csv(os.path.join(work_dir, "training_results.csv"))

# Plot the training results
training_result_directory = os.path.join(work_dir, "training_result")
if not os.path.exists(training_result_directory):
    os.mkdir(training_result_directory)
else:
    for filename in os.listdir(training_result_directory):
        file_path = os.path.join(training_result_directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

plot_and_save_push_plots(df_training, num_trials, training_result_directory)

# Evaluation

In [ ]:
# Main PETS loop
num_test_trials = 10
all_rewards = []
evaluation_result = []
goal_reached = []
plan_time = 0.0
train_time = 0.0
save_vid = True
render = True

if save_vid:
    record_every_n_frames = 1
    render_img = env.render(mode="rgb_array")
    render_img_size = (render_img.shape[1], render_img.shape[0])
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(
        os.path.join(work_dir, "evaluated_policy.mp4"),
        fourcc,
        24.0,
        render_img_size,
    )

for trial in range(num_test_trials):
    obs = env.reset()    
    agent.reset()
    
    done = False
    trial_reward = 0.0
    trial_pb_steps = 0.0
    steps_trial = 0

    tcp_pos_workframe, _, _, _, _ = env.robot.arm.get_current_TCP_pos_vel_workframe()
    cur_obj_pos_workframe = get_states_from_obs(obs)
    evaluation_result.append(np.hstack([trial, 
                                        steps_trial, 
                                        trial_pb_steps,
                                        tcp_pos_workframe, 
                                        cur_obj_pos_workframe, 
                                        env.goal_pos_workframe, 
                                        trial_reward, 
                                        False,
                                        done]))
    while not done:

        # --- Doing env step using the agent and adding to model dataset ---
        start_plan_time = time.time()
        action = agent.act(obs, **{})
        next_obs, reward, done, info = env.step(action)
        plan_time = time.time() - start_plan_time

        if render:
            render_img = env.render(mode="rgb_array")
        else:
            render_img = None
        
        obs = next_obs
        trial_reward += reward
        trial_pb_steps += info["num_of_pb_steps"]
        steps_trial += 1

        tcp_pos_workframe, _, _, _, _ = env.robot.arm.get_current_TCP_pos_vel_workframe()
        cur_obj_pos_workframe = get_states_from_obs(obs)
        evaluation_result.append(np.hstack([trial, 
                                            steps_trial, 
                                            trial_pb_steps * env._sim_time_step,
                                            tcp_pos_workframe, 
                                            cur_obj_pos_workframe, 
                                            env.goal_pos_workframe, 
                                            trial_reward, 
                                            info["tip_in_contact"],
                                            done]))
            
         # use record_every_n_frames to reduce size sometimes
        if save_vid and steps_trial % record_every_n_frames == 0:

            # warning to enable rendering
            if render_img is None:
                sys.exit('Must be rendering to save video')

            render_img = cv2.cvtColor(render_img, cv2.COLOR_BGR2RGB)
            out.write(render_img)

        if steps_trial == trial_length:
            break
    
    print("Terminated at step {} with reward {}, goal reached: {}".format(steps_trial, trial_reward, env.single_goal_reached))
    all_rewards.append(trial_reward)

    # save goal reached data during training
    if env.single_goal_reached:
        goal_reached.append(trial_reward)
    else:
        goal_reached.append(0)

if save_vid:
    out.release()

print("The average reward over {} episodes is {}".format(num_test_trials, np.mean(all_rewards)))

# Save data 
evaluation_result = np.array(evaluation_result)
df_evaluation = pd.DataFrame(evaluation_result, columns = data_columns)
pd.DataFrame(evaluation_result).to_csv(os.path.join(work_dir, "evaluation_results.csv"))

# plot evaluation results
evaluation_result_directory = os.path.join(work_dir, "evaluation_result")
if not os.path.exists(evaluation_result_directory):
    os.mkdir(evaluation_result_directory)
else:
    for filename in os.listdir(evaluation_result_directory):
        file_path = os.path.join(evaluation_result_directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

plot_and_save_push_plots(df_evaluation, num_test_trials, evaluation_result_directory)

# Plot evaluation results
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(all_rewards, 'bs-', goal_reached, 'rs')
ax.set_xlabel("Trial")
ax.set_ylabel("Trial reward")
fig.savefig(os.path.join(work_dir, "evaluation_output.png"))


# Check model learning and rollout predictions to see if code is correct

In [ ]:
# Test optimisation iterations for CEM
train_losses = []
val_scores = []

# Create a 1-D dynamics model for this environment
dynamics_model = common_util.create_one_dim_tr_model(cfg, obs_shape, act_shape)

# Create a gym-like environment to encapsulate the model
model_env = models.ModelEnvPushing(env, dynamics_model, termination_fn=None, reward_fn=None, generator=generator)

replay_buffer = common_util.create_replay_buffer(cfg, obs_shape, act_shape, rng=rng)
common_util.rollout_agent_trajectories(
    env,
    buffer_size, # initial exploration steps
    planning.RandomAgent(env),
    {}, # keyword arguments to pass to agent.act()
    replay_buffer=replay_buffer,
    trial_length=trial_length
)

print("# samples stored", replay_buffer.num_stored)

# Train model first
model_trainer = models.ModelTrainer(dynamics_model, optim_lr= 1e-3, weight_decay=5e-5)
dynamics_model.update_normalizer(replay_buffer.get_all())
dataset_train, dataset_val = common_util.get_basic_buffer_iterators(
    replay_buffer,
    batch_size=cfg.overrides.model_batch_size,
    val_ratio=cfg.overrides.validation_ratio,
    ensemble_size=ensemble_size,
    shuffle_each_epoch=True,
    bootstrap_permutes=False,  # build bootstrap dataset using sampling with replacement
)

start_train_time = time.time()
model_trainer.train(
    dataset_train, 
    dataset_val=dataset_val, 
    num_epochs=100, 
    patience=50, 
    callback=train_callback,
    silent=True)
train_time = time.time() - start_train_time

print("Training time: ", train_time)
print("Train Loss: {}, Val Loss: {}".format(train_losses[-1], val_scores[-1]))

fig, ax = plt.subplots(2, 1, figsize=(12, 10))
ax[0].plot(train_losses)
ax[0].set_xlabel("Total training epochs")
ax[0].set_ylabel("Training loss (avg. NLL)")
ax[0].xaxis.label.set_color('white')
ax[0].yaxis.label.set_color('white')
ax[0].tick_params(axis='x', colors='white')
ax[0].tick_params(axis='y', colors='white')
ax[1].plot(val_scores)
ax[1].set_xlabel("Total training epochs")
ax[1].set_ylabel("Validation score (avg. MSE)")
ax[1].xaxis.label.set_color('white')
ax[1].yaxis.label.set_color('white')
ax[1].tick_params(axis='x', colors='white')
ax[1].tick_params(axis='y', colors='white')
plt.show()

In [ ]:
# -------- Test model one set of action sequences from buffer to see exploding -------
# states still occur

planning_horizon = 10

# Get action sequence from buffer
data = replay_buffer.get_all()
action_sequences = data.act[0:planning_horizon,:]
action_sequences = np.tile(action_sequences, (5,1,1)).astype(np.float32)
action_sequences = torch.from_numpy(action_sequences)
# print(action_sequences.shape)

# Initialise state and create model input
initial_state = data.obs[0]
# print(initial_state.shape)
initial_obs_batch = np.tile(initial_state, (5,1)).astype(np.float32)
# print(initial_obs_batch.shape)
model_state = model_env.reset(initial_obs_batch, return_as_np=False)
# print(model_state['propagation_indices'])

batch_size = initial_obs_batch.shape[0]
total_rewards = torch.zeros(batch_size, 1)
terminated = torch.zeros(batch_size, 1, dtype=bool)
model_env.reset_batch_goals(batch_size)

print(data.obs[1][0:3])
print(data.next_obs[1][0:3])
print(data.act[1])

for time_step in range(planning_horizon):
    print(torch.mean(model_state["obs"], 0)[0:3])
    # print(model_state["obs"].shape)
    # print(torch.mean(model_state["obs"]))
    action_for_step = action_sequences[:, time_step, :]
    # print(action_for_step[0])

    # Re-initialise model state from data buffer with every time step (1 step rollouts)
    # Comment out to do planning_horizon step rollouts
    # initial_state = data.obs[time_step]
    # initial_obs_batch = np.tile(initial_state, (5,1)).astype(np.float32)
    # initial_obs_batch = torch.from_numpy(initial_obs_batch)
    # model_state.update({'obs': initial_obs_batch})
    # action_batch = torch.repeat_interleave(
    #     action_for_step, 20, dim=0
    # )

    action_batch = action_for_step
    # ---------------- Use model_env.step -----------------
    # _, rewards, dones, model_state = model_env.step(
    #     action_batch, model_state, sample=True
    # )
    # rewards[terminated] = 0
    # terminated |= dones
    # total_rewards += rewards

    # -------------- Use one_dim_tr_model sample -------------
    # with torch.no_grad():
    #     next_observs, _, _, next_model_state, = model_env.dynamics_model.sample(
    #         action_batch, model_state, deterministic=False, rng=model_env._rng,
    #     )

    # -------------- Use model.sample_1d() --------------------
    # with torch.no_grad():
    #     obs = model_state["obs"]
    #     model_in = model_env.dynamics_model._get_model_input(model_state["obs"], action_batch)
    #     next_observs, _ = model_env.dynamics_model.model.sample_1d(
    #         model_in, model_state, rng=model_env._rng, deterministic=False
    #     )
    #     next_observs += obs
    #     model_state["obs"] = next_observs

    # -------------- Use model.forward()-------------------------
    with torch.no_grad():
        obs = model_state["obs"]
        model_in = model_env.dynamics_model._get_model_input(model_state["obs"], action_batch)
        means, logvars = model_env.dynamics_model.model.forward(
            model_in, rng=model_env._rng, propagation_indices=model_state["propagation_indices"]
        )
        variances = logvars.exp()
        stds = torch.sqrt(variances)
        # stds = torch.ones((5,30))
        next_observs = torch.normal(means, stds, generator=model_env._rng)
        # next_observs = means
        # print(torch.mean(means))
        # print(torch.mean(logvars))
        # print(torch.mean(stds))
        if dynamics_model.target_normalizer:
            next_observs = dynamics_model.target_normalizer.denormalize(next_observs)

        if dynamics_model.target_is_delta:
            next_observs += obs
        model_state["obs"] = next_observs

In [ ]:
# evaluation  function
def evaluate(env, agent, training_trial_num, work_dir):
    
    all_rewards = []
    evaluation_result = []
    goal_reached = []
    plan_time = 0.0
    train_time = 0.0
    save_vid = True
    render = True

    if save_vid:
        record_every_n_frames = 1
        render_img = env.render(mode="rgb_array")
        render_img_size = (render_img.shape[1], render_img.shape[0])
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(
            os.path.join(work_dir, "evaluated_policy_training_trial_{}.mp4".format(training_trial_num)),
            fourcc,
            24.0,
            render_img_size,
        )

    for trial in range(1):
        obs = env.reset()    
        agent.reset()
        
        done = False
        trial_reward = 0.0
        trial_pb_steps = 0.0
        steps_trial = 0

        while not done:

            # --- Doing env step using the agent and adding to model dataset ---
            start_plan_time = time.time()
            action = agent.act(obs, **{})
            next_obs, reward, done, info = env.step(action)
            plan_time = time.time() - start_plan_time

            if render:
                render_img = env.render(mode="rgb_array")
            else:
                render_img = None
            
            obs = next_obs
            trial_reward += reward
            trial_pb_steps += info["num_of_pb_steps"]
            steps_trial += 1

                
            # use record_every_n_frames to reduce size sometimes
            if save_vid and steps_trial % record_every_n_frames == 0:

                # warning to enable rendering
                if render_img is None:
                    sys.exit('Must be rendering to save video')

                render_img = cv2.cvtColor(render_img, cv2.COLOR_BGR2RGB)
                out.write(render_img)

            if steps_trial == trial_length:
                break
        
        all_rewards.append(trial_reward)

        # save goal reached data during training
        if env.single_goal_reached:
            goal_reached.append(trial_reward)
        else:
            goal_reached.append(0)

    if save_vid:
        out.release()

# create a evaluation environment
eval_env = gym.make(env_name, **env_kwargs)
eval_env.seed(seed)